# Assignment: Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`
### Do Q2, and one of Q1 or Q3.

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? 
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [68]:
# 1.
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

# Create data frame from the csv file
df = pd.read_csv('./data/airbnb_hw.csv')

# Remove any dollar signs, commas, or spaces in the data
df['Price'] = df['Price'].replace('\$', '', regex=True)
df['Price'] = df['Price'].replace(',', '', regex=True)
df['Price'] = df['Price'].replace(' ', '', regex=True)

# Convert Price variable to numeric values
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Check for null/missing values
print('Missing values:', sum(df['Price'].isnull()))

# In the end we end up with zero missing values

Missing values: 0


In [ ]:
# 2.
# Create data frame from the csv file
mn = pd.read_csv('./data/mn_police_use_of_force.csv')

# Count missing values and yes/no values then find the proportion
nan = sum(mn['subject_injury'].isnull())
yn = mn['subject_injury'].count()
prop = nan / (nan + yn)
print(prop) # We see that 76% of all entries into the subject injury variable are missing - This is a major concern since it means police are not reporting injuries

# Since there are so many missing values I am going to make the nan values their own variable; the fact that they are missing gives us an important insight so it should be a value
mn['subject_injury'] = mn['subject_injury'].replace(np.nan, 'None', regex=True)
print(mn['subject_injury'].unique())
print(mn['subject_injury'].value_counts())

# Cross tabulate subject_injury with the force_type variable
pd.crosstab(mn['subject_injury'], mn['force_type'])

# We see that a large proportion of cases that involve chemical irritants, bodily force and taser tend to have missing values
# While this is also true for the less lethal projectile and maximal restraint technique columns, we see that they only have missing values which implies these cases may be assumed to cause no injury or some other outside reason, so we will not look too into these

0.7619342359767892
['None' 'No' 'Yes']
subject_injury
None    9848
Yes     1631
No      1446
Name: count, dtype: int64


force_type,Baton,Bodily Force,Chemical Irritant,Firearm,Gun Point Display,Improvised Weapon,Less Lethal,Less Lethal Projectile,Maximal Restraint Technique,Police K9 Bite,Taser
subject_injury,,,,,,,,,,,
No,0,1093,131,2,33,34,0,1,0,2,150
None,2,7051,1421,0,27,74,87,0,170,31,985
Yes,2,1286,41,0,44,40,0,2,0,44,172


In [87]:
# 3.

# Import data
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df = pd.read_csv(url)

# Check for unique values and count them up
print(df['WhetherDefendantWasReleasedPretrial'].unique())
print(df['WhetherDefendantWasReleasedPretrial'].value_counts())

# Replace 9 with np.nan, assuming 9 is a missing value
df['WhetherDefendantWasReleasedPretrial'] = df['WhetherDefendantWasReleasedPretrial'].replace(9, np.nan, regex=True)

# Check for unique values to make sure that the missing values were correctly replaced
print(df['WhetherDefendantWasReleasedPretrial'].unique())

/tmp/ipykernel_3205/670556606.py:5: DtypeWarning: Columns (1,4,5,7,79,80,81,82,83,84,108,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,270,271,272,273,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,301,302,303,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398

[9 0 1]
WhetherDefendantWasReleasedPretrial
1    19154
0     3801
9       31
Name: count, dtype: int64
[nan  0.  1.]


In [115]:
# 4. 
# Import data
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df = pd.read_csv(url)

print(df['ImposedSentenceAllChargeInContactEvent'].unique().sum())
print(df['SentenceTypeAllChargesAtConvictionInContactEvent'].unique())

/tmp/ipykernel_3205/904032260.py:4: DtypeWarning: Columns (1,4,5,7,79,80,81,82,83,84,108,163,164,165,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,270,271,272,273,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,301,302,303,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398

 6012.985626283367556366245.913757700205341207211.991786447638602.9568788501026784108300240180496254.328542094455852445115132482583476.164271047227926.131416837782341111931.9712525667351136.9856262833676.065708418891170435.4928131416838106.49281314168483518.314168377823448032932347321.164271047227934.65708418891172174.492813141683781860043.164271047227917952302019270214555311.905544147843911435.006160164271168.65708418891170446.6242299794661102652005724.328542094455912.164271047227911781.492813141683822.492813141683819803.657084188911756102.7926078028747414722150040284111184216215647.295687885010310551246297532436034.4804928131417120.32854209445659.92607802874746659.991786447638666051.164271047227914.95687885010273.98562628336756782281.47843942505133624.886168233348.032854209445672034812002749874206379.926078028747457.034907597535949.971252566735159.492813141683817238.49281314168460.98562628336761264515821622742.956878850102744570.952772073922516177.8213552361417529010801414.8295687885

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q4.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?